In [1]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [47]:
!pip install preprocessor
!pip install tweet-preprocessor
!pip install beautifulsoup4
!pip install fasttext
!pip install emoji
import nltk
nltk.download('stopwords')

     |████████████████████████████████| 170 kB 31.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=7b6d31e6f87a9014e577e76f45f02e12e5f71a6804a44cb0ecc3b299010f81bf
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import preprocessor
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import fasttext
import csv
import datetime
from bs4 import BeautifulSoup
import re
import itertools
import emoji

%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [49]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import os
import json
import argparse
import importlib
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from sklearn import metrics

def getData(type):

    folder_path = "/content/drive/MyDrive/CS640-project/DataTwitter/"+ type + "/"
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.endswith('txt'):
            path = os.path.join(folder_path, filename)
            df = pd.read_csv(path, sep="\\t", header=None)

            df.rename(columns={1: "tweet", 2: "sentiment"}, inplace=True)
            df = df.drop(columns=[df.columns[0], df.columns[3]])
            df['sentiment'] = df['sentiment'].replace(['anger'],0)
            df['sentiment'] = df['sentiment'].replace(['fear'],1)
            df['sentiment'] = df['sentiment'].replace(['joy'],2)
            df['sentiment'] = df['sentiment'].replace(['sadness'],3)

            dfs.append(df)


    df = pd.concat(dfs)
    # df.columns = ['text', 'label']
    # df['label_numeric'] = df['label'].astype('category').cat.codes
    print('total ' + type +' samples : ', len(df))
    print(df['sentiment'].value_counts())

    return df




train = getData("Train")
dev = getData("Dev")
test = getData("Test")
# train = pd.concat([train, test])
# train = train.sample(frac=0.01, replace=True, random_state=1)
# # test = test.sample(frac=0.1, replace=True, random_state=1)
X_train = train.filter(regex='tweet').tweet.values
y_train = train.filter(regex='sentiment').sentiment.values
X_dev = dev.filter(regex='tweet').tweet.values
y_dev = dev.filter(regex='sentiment').sentiment.values
X_test = test.filter(regex='tweet').tweet.values
y_test = test.filter(regex='sentiment').sentiment.values







total Train samples :  3613
1    1147
0     857
2     823
3     786
Name: sentiment, dtype: int64
total Dev samples :  347
1    110
0     84
2     79
3     74
Name: sentiment, dtype: int64
total Test samples :  3142
1    995
0    760
2    714
3    673
Name: sentiment, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [52]:
import os
import json
import argparse
import importlib
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from sklearn import metrics

def getData():

    folder_path = "/content/drive/MyDrive/CS640-project/DataInstagram/"
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.endswith('csv'):
            path = os.path.join(folder_path, filename)
            df = pd.read_csv(path)

            df.rename(columns={'imagename': "image_id", "Contents": "tweet", 'Q5_presence_asian': "isEastAsian"}, inplace=True)
            df = df.drop(columns=[df.columns[1], df.columns[2], df.columns[3], df.columns[4], df.columns[6], df.columns[7], df.columns[8], df.columns[9],
                                  df.columns[10], df.columns[11], df.columns[13], df.columns[14], df.columns[15], df.columns[16], df.columns[17],
                                  df.columns[18], df.columns[19], df.columns[20], df.columns[21], df.columns[22], df.columns[23], df.columns[24], df.columns[25]])
            # df['sentiment'] = df['sentiment'].replace(['anger'],0)
            # df['sentiment'] = df['sentiment'].replace(['fear'],1)
            # df['sentiment'] = df['sentiment'].replace(['joy'],2)
            # df['sentiment'] = df['sentiment'].replace(['sadness'],3)

            dfs.append(df)


    df = pd.concat(dfs)
    # df.columns = ['text', 'label']
    # df['label_numeric'] = df['label'].astype('category').cat.codes
    print('total ' + "insta" +' samples : ', len(df))
    # print(df['isEastAsian'].value_counts())

    return df




Insta_train = getData().tweet.values
print(Insta_train)

# train = pd.concat([train, test])
# train = train.sample(frac=0.01, replace=True, random_state=1)
# # test = test.sample(frac=0.1, replace=True, random_state=1)
# Insta_train = train.filter(regex='tweet').tweet.values
# y_train = train.filter(regex='sentiment').sentiment.values








total insta samples :  9648
['#covid #covid2020 #covidvirus #virus #coronavairus #coronavirus #coronavírus #coronavirüs #blackandwhite #blackandwhiteportrait #blackandwhitephoto #blackandwhite_photos #lockdown #lockdown2020 #lockdownlife #lockdownitaly #italylockdown #lockdowndiaries #lockdownactivities #stayathome #staysafe #stayhome #iorestoacasa #myhome #covid19 #covıd19 #covi̇d_19 #coviditalia #🖤 #🖤🖤🖤'
 'Well this is the final mural of my trip in Australia, a very weird trip, to be honest I couldn’t connect with my painting, at the beginning it was a popular psicosis which looked unreal, then the airline call me with news that my flights was rebooked for 4 month later, I still had a lot to do, people to meet and paint to make, it was super sad when I had to buy another thicket a week before of planed and run out of the country, the same day they closed the border, I like to think that everything happens for reason, everything is meant to be ... if this is my way to start the painti

In [35]:
# def text_preprocessing(text):  
#     text = re.sub(r'(@.*?)[\s]', '', text)
#     text = re.sub(r'&amp;', '&', text)
#     text = re.sub(r"<[\w'/'\s]*>",'' ,text)
#     text = re.sub(r"[\'\"\-]+", '',text)
#     text = re.sub(r"@[\w]+", '',text)
#     text = re.sub(r'http\S+', '',text)
#     text = re.sub(r'\s+', '', text).strip()
#     return text
preprocessor.set_options(preprocessor.OPT.URL,preprocessor.OPT.RESERVED)
stop_words = nltk.corpus.stopwords.words('english')
def text_preprocessing(text):   
    text = preprocessor.tokenize(text)
    text = ' '.join([word for word in text.split(' ') if word.lower() not in stop_words])
    return text

In [58]:
lemmatizer = WordNetLemmatizer()

def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }


def load_dict_contractions_slangs():
    
    cont = {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "i'd":"i would",
        "i'll":"i will",
        "i'm":"i am",
        "i'm'a":"i am about to",
        "i'm'o":"i am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "i've":"i have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks",
        "rn":"right now",
        "atm": "at the moment",
        "idk": "i dont know",
        "cuz": "because",
        "bcuz": "because",
        "ur": "you are",
        "ly": "love you",
        "lol": "laugh out loud",
        "rofl": "rolling on floor laughing",
        "lmao": "laughing my ass off",
        "ok": "okay",
        "ty": "thank you",
        "fav": "favorite",
        "omg": "oh my god"
        }
    
    to_ret = {}
    for k,v in cont.items():
        key = k.lower().replace("'",'')
        to_ret[key] = v.lower()
        
    return to_ret


def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)


def tweet_cleaning_for_sentiment_analysis(tweet):    
    
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    #Removal of account
    tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)", " ", tweet).split())
    # removal of hashtag
    tweet = tweet.replace('#','')
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    #Lower case
    tweet = tweet.lower()
    # rempval of 'RT'
    tweet = tweet.replace('rt','')
    # Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    # removal of stop words
    tweet = ' '.join([word for word in tweet.split(' ') if word not in stop_words])
    # CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS_SLANGS = load_dict_contractions_slangs()
    tweet = tweet.replace("’","'").replace("'","")
    words = tweet.split()
    reformed = [CONTRACTIONS_SLANGS[word] if word in CONTRACTIONS_SLANGS else word for word in words]
    tweet = " ".join(reformed)

    # Standardizing words - lemmatization
    tweet = ' '.join([lemmatizer.lemmatize(word) for word in tweet.split(' ')]).lower()
    
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    # replace emojis with desctiption - remove accents - remove underscores
    tweet = emoji.demojize(tweet).lower()
    tweet = strip_accents(tweet).lower()
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split()).replace('_',' ')
    
    # remove new line characters
    tweet = tweet.strip().replace('\\n',' ')
    
    # only keep alphabets
    tweet = ''.join([c for c in tweet if (c.isalpha() or c ==' ')]).strip()
    
    # rempove double spaces and triple spaces
    tweet = tweet.replace('   ',' ').replace('  ',' ')
    
    # one word replacements
    replacement_dict = {
        'u': 'you',
        'v': 'we',
        'r': 'are',
        'w': 'we',
        'n': 'and',
        'nd': 'and',
        '&': 'and'
    }
    
    tweet = ' '.join([replacement_dict[word] if word in replacement_dict else word for word in tweet.split(' ')])
    
    # removal of stop words
    tweet = ' '.join([word for word in tweet.split(' ') if word not in stop_words])

    return tweet
    

In [64]:
def preprocessAllData(X):
  count = []
  res = []
  for data in X:
    if data != "":
      try:
        res.append(tweet_cleaning_for_sentiment_analysis(data))
        count.append(len(res))
      except:
        #nan values are filtered here 
        pass
  print(count)
  return np.array(res)

# X_train = tweet_cleaning_for_sentiment_analysis(X_train)
# X_dev = preprocessAllData(X_dev)
# X_test = preprocessAllData(X_test)
# X_insta = preprocessAllData(Insta_train)

train['tweet'] = train['tweet'].apply(tweet_cleaning_for_sentiment_analysis)
dev['tweet'] = dev['tweet'].apply(tweet_cleaning_for_sentiment_analysis)
test['tweet'] = test['tweet'].apply(tweet_cleaning_for_sentiment_analysis)
X_train = train.filter(regex='tweet').tweet.values
y_train = train.filter(regex='sentiment').sentiment.values
X_dev = dev.filter(regex='tweet').tweet.values
y_dev = dev.filter(regex='sentiment').sentiment.values
X_test = test.filter(regex='tweet').tweet.values
y_test = test.filter(regex='sentiment').sentiment.values



Naive Bayes 

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
num_features = 3000
#count vectorizer 
cv = CountVectorizer(max_features=num_features)
cv.fit_transform(X_train)
X_train_cv = cv.transform(X_train)
X_dev_cv = cv.transform(X_dev)
X_test_cv = cv.transform(X_test)

#tfidf vectorizer 

tfidf = TfidfVectorizer(max_features=num_features)
tfidf.fit_transform(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_dev_tfidf = tfidf.transform(X_dev)
X_test_tfidf = tfidf.transform(X_test)


#word2vec

# word2vecModel = Word2Vec(X_train, min_count=1,size= 50,workers=3, window =3, sg = 1)
# X_train_word2vec = []
# for i in X_train:
#   X_train_word2vec.append(word2vecModel[i])
# print(X_train_word2vec)



In [74]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model_cv = MultinomialNB()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_cv.predict(cv.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())


model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_tfidf.predict(tfidf.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

              precision    recall  f1-score   support

           0       0.79      0.75      0.77        84
           1       0.80      0.69      0.74       110
           2       0.77      0.84      0.80        79
           3       0.72      0.84      0.77        74

    accuracy                           0.77       347
   macro avg       0.77      0.78      0.77       347
weighted avg       0.77      0.77      0.77       347

count vectorizer score:  0.7694524495677233
instagram data results: 
Series([], dtype: int64)
              precision    recall  f1-score   support

           0       0.85      0.75      0.80        84
           1       0.77      0.78      0.77       110
           2       0.78      0.80      0.79        79
           3       0.79      0.85      0.82        74

    accuracy                           0.79       347
   macro avg       0.80      0.80      0.79       347
weighted avg       0.79      0.79      0.79       347

tfidf vectorizer score:  0.806916426

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model_cv = LogisticRegression()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_cv.predict(cv.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())


model_tfidf = LogisticRegression()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_tfidf.predict(tfidf.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())


              precision    recall  f1-score   support

           0       0.84      0.83      0.84        84
           1       0.81      0.84      0.82       110
           2       0.86      0.85      0.85        79
           3       0.83      0.81      0.82        74

    accuracy                           0.83       347
   macro avg       0.84      0.83      0.83       347
weighted avg       0.83      0.83      0.83       347

count vectorizer score:  0.8328530259365994
instagram data results: 
Series([], dtype: int64)
              precision    recall  f1-score   support

           0       0.86      0.80      0.83        84
           1       0.83      0.79      0.81       110
           2       0.83      0.85      0.84        79
           3       0.80      0.89      0.84        74

    accuracy                           0.83       347
   macro avg       0.83      0.83      0.83       347
weighted avg       0.83      0.83      0.83       347

tfidf vectorizer score:  0.838616714

In [76]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

model_cv = LinearSVC()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_cv.predict(cv.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())


model_tfidf = LinearSVC()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_tfidf.predict(tfidf.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

              precision    recall  f1-score   support

           0       0.85      0.82      0.84        84
           1       0.81      0.86      0.84       110
           2       0.82      0.82      0.82        79
           3       0.81      0.77      0.79        74

    accuracy                           0.82       347
   macro avg       0.83      0.82      0.82       347
weighted avg       0.82      0.82      0.82       347

count vectorizer score:  0.8242074927953891
instagram data results: 
Series([], dtype: int64)
              precision    recall  f1-score   support

           0       0.91      0.82      0.86        84
           1       0.85      0.80      0.83       110
           2       0.81      0.87      0.84        79
           3       0.80      0.89      0.84        74

    accuracy                           0.84       347
   macro avg       0.84      0.85      0.84       347
weighted avg       0.85      0.84      0.84       347

tfidf vectorizer score:  0.864553314

In [77]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

model_cv = MLPClassifier()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_cv.predict(cv.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())


model_tfidf = MLPClassifier()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_tfidf.predict(tfidf.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

              precision    recall  f1-score   support

           0       0.86      0.85      0.85        84
           1       0.82      0.79      0.81       110
           2       0.81      0.85      0.83        79
           3       0.79      0.80      0.79        74

    accuracy                           0.82       347
   macro avg       0.82      0.82      0.82       347
weighted avg       0.82      0.82      0.82       347

count vectorizer score:  0.8184438040345822
instagram data results: 
Series([], dtype: int64)
              precision    recall  f1-score   support

           0       0.84      0.85      0.84        84
           1       0.84      0.74      0.79       110
           2       0.80      0.89      0.84        79
           3       0.76      0.80      0.78        74

    accuracy                           0.81       347
   macro avg       0.81      0.82      0.81       347
weighted avg       0.81      0.81      0.81       347

tfidf vectorizer score:  0.772334293

OPEN AI GPT-3

In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model_cv = RandomForestClassifier()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_cv.predict(cv.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())


model_tfidf = RandomForestClassifier()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
for row in X_insta:
  naiveBayesInstaPredictions.append(model_tfidf.predict(tfidf.transform([row]))[0])
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

              precision    recall  f1-score   support

           0       0.93      0.81      0.87        84
           1       0.80      0.93      0.86       110
           2       0.87      0.86      0.87        79
           3       0.87      0.80      0.83        74

    accuracy                           0.86       347
   macro avg       0.87      0.85      0.86       347
weighted avg       0.86      0.86      0.86       347

count vectorizer score:  0.8559077809798271
instagram data results: 
Series([], dtype: int64)
              precision    recall  f1-score   support

           0       0.83      0.80      0.81        84
           1       0.82      0.85      0.83       110
           2       0.86      0.85      0.85        79
           3       0.80      0.81      0.81        74

    accuracy                           0.83       347
   macro avg       0.83      0.83      0.83       347
weighted avg       0.83      0.83      0.83       347

tfidf vectorizer score:  0.838616714